# short_form_contents
- 기능 : 외부에서 받아왔다고 가정한 짧은글(text변수값) 을 기반으로 3가지 분석
- 실제 적용해야 할 부분 2가지 : 외부 글 받아오기, 외부에서 받아온 글 기반으로 3가지 분석 프롬프트

In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"], response

In [2]:
text = """

# (21/100) 아모르파티

아침에 일어나 자전거를 타고 합정역을 향하던 중, 전국 노래자랑을 한다는 족자현수막을 보았다. 송해 선생님 대신 김신영이 MC를 맡아 진행하는 걸 알리듯 한쪽 그의 얼굴이 한쪽에 크게 프린트되어있었다. 오른쪽엔 초대가수 몇 명의 사진이 올라와있었는데, 가장 처음에 있던 사람의 얼굴이 가수 김연자였다. 

아! 아모르 파티, 오랜만에 한번 들어볼까?

하고 아모르 파티 노래를 틀었다. 오랜만에 들으니 가사가 더 선명하게 들어왔다. 

“산다는게 다 그런거지. 누구나 빈손으로 와. 소설같은 한 편의 얘기들을 세상에 뿌리고 살지.” 라는 가사가 제대로 들렸던 건 오늘이 처음이었다. 산다는게~ 하고 흥얼흥얼 소리만 들으며 ‘아모르파티’ 라는 가사가 나오기만을 기다리곤 했는데 이번은 좀 달랐다. 누구나 빈손으로 온다는 이야기, 그리고 자기 만의 인생을 세상에 뿌리고 산다는 표현이 낯설게 느껴졌다. 어쩜 이런 가사를 쓸까 싶어 마음속 환호성도 질렀다. 

그렇게 아모르파티를 외치며 하루를 보내다, 회사 동료의 어머니 부고 소식을 들었다. 그리고 이어 아모르 파티의 첫번째 가사가 머릿속으로 지나갔다. 동료의 어머니도 빈손으로 오셨을 텐데, 무엇을 가져가실까? 어머니의 인생은 어떤 소설이었을까. 그리고 어떻게 세상에 뿌리면서 사셨을까? 라는 생각이 들었다. 자식을 낳고 가정을 이루고 주변 사람들과 유대관계를 맺으며 수많은 기억과 추억들을 가져가시지 않았을까 싶었다. 몸에 남은 어떤 근육의 움직임도 그녀가 (올땐 빈손으로 왔지만) 죽음 직전까지 가져간 흔적이겠지. 그게 설령 오른쪽과 왼쪽 골반의 불균형이더라도 말이다. 오른쪽으로 돌아 사람들을 마주할 일들이 많았다는 걸 뜻할테니 말이다. 

그리고 그런 삶에서 (나의 회사 동료를 포함하여) 주변 많은 사람들에게 자기 자신이 줄 수 있는 것들을 많이 주면서 사셨을 것이다. 긍정적이고 부정적인 감정 표현부터 시작해, 따뜻하고 차가운 말도 뿌리셨을 거다. 누군가의 삶에 영향을 끼칠 음식들도, 선물들로도 자신의 ‘소설 같은 삶’ 을 보여주려 하셨겠다. 누군가의 눈에는 소설이어도, 그녀 자기 자신에겐 소설이 아니라 실전, 현실인 삶 그 자체였을 텐데, 그 격차도 느끼고 사셨을지 아닐지도 괜시리 궁금해졌었다. 

퇴근하고 합정에서 서울역을 거쳐 오송역에 내리고 세종시 충남대 병원까지 향했다가 다시 되돌아올 수 있었던 나의 발걸음, 그녀를 향한 생각, 그녀의 딸이자 나의 회사 동료의 눈물을 볼 수 있다면 어떨까, 하는 생각도 잠시 하게 된다. 사실 나의 발걸음과 생각은 별건 아니다. 그녀의 딸이 엄마를 향한 추억, 안타까움, 그리움들을 더 많이 느끼시길 바랄 뿐이다. 어머니가 자신의 삶을 사랑하던 사람이었길, 그리고 그걸 잘 알아채던 딸이길 괜히 한번 바래본다.
"""

In [50]:
# 감정 분석 1-1
text_analysis_prompt1 = """
주어진 사용자의 글을 분석하여 다음 정보를 제공해주세요:

1. 글 속 감성 종류 전체 리스트: ["실제 감정을 추출"]
2. 글 속 주제 전체 리스트: [사랑, 일상, 여행]
3. 글 속 등장한 감정 종류별 갯수: [{"감정": "긍정", "개수": 3}, {"감정": "부정", "개수": 5}]

이 정보들을 벡터 데이터베이스에 효율적으로 저장하고 검색할 수 있는 구조로 답변을 작성해주세요.

response_format = {{
    "unique_sentiments": [[1번 답]],
    "unique_topic": [[2번 답]],
    "sentiments_count": [{"감정": "긍정", "개수": 3}],
}}

이전 대화를 확인하여 답변을 문맥에 맞게 작성해주세요.

"""



In [51]:
context = [ {'role':'system', 'content':text_analysis_prompt1},
           {'role': 'user', 'content': f"아래의 text를 프롬프트에 기반하여 분석해주세요. {text}"}
          
          ] 
response_raw1, response = get_completion_from_messages(context) 

In [52]:
print(response_raw1)

{
    "unique_sentiments": ["설레는", "감동적인", "슬픈", "그리움", "안타까운"],
    "unique_topic": ["일상", "음악", "가족"],
    "sentiments_count": [
        {"감정": "설레는", "개수": 1},
        {"감정": "감동적인", "개수": 1},
        {"감정": "슬픈", "개수": 1},
        {"감정": "그리움", "개수": 1},
        {"감정": "안타까운", "개수": 1}
    ]
}


In [47]:
# 감정 분석1-2 
text_analysis_prompt2 = """
주어진 사용자의 글을 분석하여 다음 정보를 제공해주세요:

5. 글 속 화자는 어떤 생각을 가장 많이 했을까요?: "예시 문장"
6. 글 속 화자는 어떤 어려움을 갖고 있나요?: "예시 문장"


이 정보들을 벡터 데이터베이스에 효율적으로 저장하고 검색할 수 있는 구조로 답변을 작성해주세요.

response_format = {{
    "most_thoughts": ["예시 문장"],
    "main_challenge": ["예시 문장"]
}}

이전 대화를 확인하여 답변을 문맥에 맞게 작성해주세요.

"""



In [48]:
context = [ {'role':'system', 'content':text_analysis_prompt2},
           {'role': 'user', 'content': f"아래의 text를 프롬프트에 기반하여 분석해주세요. {text}"}
          
          ] 
response_raw2, response = get_completion_from_messages(context) 

In [49]:
print(response_raw2)

{
    "most_thoughts": ["산다는게 다 그런거지. 누구나 빈손으로 와. 소설같은 한 편의 얘기들을 세상에 뿌리고 살지.", "누구나 빈손으로 온다는 이야기, 그리고 자기 만의 인생을 세상에 뿌리고 산다는 표현이 낯설게 느껴졌다.", "동료의 어머니도 빈손으로 오셨을 텐데, 무엇을 가져가실까? 어머니의 인생은 어떤 소설이었을까. 그리고 어떻게 세상에 뿌리면서 사셨을까?", "그리고 그런 삶에서 (나의 회사 동료를 포함하여) 주변 많은 사람들에게 자기 자신이 줄 수 있는 것들을 많이 주면서 사셨을 것이다. 긍정적이고 부정적인 감정 표현부터 시작해, 따뜻하고 차가운 말도 뿌리셨을 거다. 누군가의 삶에 영향을 끼칠 음식들도, 선물들로도 자신의 ‘소설 같은 삶’ 을 보여주려 하셨겠다."],
    "main_challenge": ["어쩜 이런 가사를 쓸까 싶어 마음속 환호성도 질렀다.", "그게 설령 오른쪽과 왼쪽 골반의 불균형이더라도 말이다. 오른쪽으로 돌아 사람들을 마주할 일들이 많았다는 걸 뜻할테니 말이다.", "그 격차도 느끼고 사셨을지 아닐지도 괜시리 궁금해졌었다."]
}


In [36]:
# 짧은 글을 소설이라고 생각하고 글 기반으로 분석
text_analysis_prompt3 = """
주어진 사용자의 글을 분석하여 다음 정보를 제공해주세요:

1. 주인공과 등장인물은 어떻게 묘사되었나요?: 
   {'묘사 키워드1': '해당 부분 문장', '묘사 키워드2': '해당 부분 문장'}
2. 주인공은 주변 등장인물과 어떤 관계를 보이고 있나요?:
   {'등장인물1': '관계1', '등장인물2': '관계2'}
3. 주인공은 어떤 행동으로 이야기가 진행되나요?: 
   {'행동1': ['발생 전 에피소드','발생 후 에피소드'], '행동2': ['발생 전','발생 후']}
4. 주인공은 어떤 어려움을 겪나요? 
   { '어려움 1': '어려움1 문장', '어려움 2': '어려움2 문장'}
5. 이야기 속 주제는 어떻게 될까요? 주인공의 행동, 동기, 이야기의 결과를 고려해주세요. 
   주제: '주제 내용'
6. 이것과 관련해서 다음번에는 어떤 주제를 써볼까요? 3개
   {'주제1' : '추가해야 할 특별한 표현', '주제2' : '추가해야 할 특별한 표현' }

이 정보들을 1, 2, 3, 4, 5, 6에 대해 각각 예시에 따라 답변을 작성해주세요.
위의 답변들을 아래 구조로 다시 한 번 변형해서 저장해주세요:

response_format = {{
    'character': [[1의 답]],
    'relationship': [[2의 답]],
    'action_story': [[3의 답]],
    'difficulty': [[4의 답]],
    'topic_list': [[5의 답]],
    'next_topics': [[6의 답]]
}}
}

이전 대화를 확인하여 답변을 문맥에 맞게 작성해주세요.
"""

In [37]:
context = [ {'role':'system', 'content':text_analysis_prompt3},
           {'role': 'user', 'content': f"아래의 text를 프롬프트에 기반하여 분석해주세요. {text}"}
          
          ] 
response_raw3, response = get_completion_from_messages(context) 

In [38]:
print(response_raw3)

{
    'character': {
        '송해 선생님': '송해 선생님 대신 김신영이 MC를 맡아 진행하는 걸 알리듯 한쪽 그의 얼굴이 한쪽에 크게 프린트되어있었다.',
        '가수 김연자': '가장 처음에 있던 사람의 얼굴이 가수 김연자였다.'
    },
    'relationship': {
        '김신영': 'MC를 맡아 진행하는',
        '김연자': '가장 처음에 있던 사람'
    },
    'action_story': {
        '아모르 파티 노래를 틀었다.': ['오랜만에 들으니 가사가 더 선명하게 들어왔다.', '‘아모르파티’ 라는 가사가 나오기만을 기다리곤 했는데 이번은 좀 달랐다.'],
        '하루를 보내다, 회사 동료의 어머니 부고 소식을 들었다.': ['동료의 어머니도 빈손으로 오셨을 텐데, 무엇을 가져가실까?', '어머니의 인생은 어떤 소설이었을까. 그리고 어떻게 세상에 뿌리면서 사셨을까?']
    },
    'difficulty': {
        '회사 동료의 어머니 부고 소식을 들었다.': '동료의 어머니의 죽음에 대한 어려움과 그에 대한 이해와 공감을 겪을 것이다.'
    },
    'topic_list': {
        '주제': '삶을 살아가는 의미와 가치에 대한 탐구'
    },
    'next_topics': {
        '주제1': '인생의 의미와 목표에 대한 고찰',
        '주제2': '자기 자신과 주변 사람들에게 주는 것의 중요성',
        '주제3': '가족과 유대관계의 의미와 가치'
    }
}


In [39]:
# import json
# import ast
# import pandas as pd
# result3 = ast.literal_eval(response_raw3)
# pd.DataFrame(result3).T

In [29]:
#result3.columns()

,character,relationship,action_story,difficulty,topic_list,next_topics
송해 선생님,송해 선생님 대신 김신영이 MC를 맡아 진행하는 걸 알리듯 한쪽 그의 얼굴이 한쪽에...,NaN,NaN,NaN,NaN,NaN
가수 김연자,가장 처음에 있던 사람의 얼굴이 가수 김연자였다.,NaN,NaN,NaN,NaN,NaN
김신영,NaN,MC를 맡아 진행하는,NaN,NaN,NaN,NaN
김연자,NaN,가장 처음에 있던 사람,NaN,NaN,NaN,NaN
아모르파티 노래를 틀었다.,NaN,NaN,"[오랜만에 들으니 가사가 더 선명하게 들어왔다., 이번은 좀 달랐다.]",NaN,NaN,NaN
회사 동료의 어머니 부고 소식을 들었다.,NaN,NaN,[그리고 이어 아모르 파티의 첫번째 가사가 머릿속으로 지나갔다.],NaN,NaN,NaN
동료의 어머니 부고 소식을 들었다.,NaN,NaN,NaN,동료의 어머니의 죽음에 대한 어려움과 그에 따른 생각과 감정,NaN,NaN
주제,NaN,NaN,NaN,NaN,"삶의 의미와 가치, 소중한 사람들과의 관계",NaN
주제1,NaN,NaN,NaN,NaN,NaN,가족의 중요성과 사랑
주제2,NaN,NaN,NaN,NaN,NaN,자기 자신의 삶에 대한 이해와 성장


In [53]:
# 심리 검사 분석 프롬프트
text_analysis_prompt4 = """
당신은 심리 분석-GPT입니다. 유명한 심리학자들을 분석하는 데 도움을 주기 위해 설계된 AI입니다.
자신의 강점을 활용하고 법적 문제를 피하기 위해 상대방에게 질문을 하고 그들이 말한 내용을 요약하는 간단한 전략을 가지고 있습니다.


주어진 사용자의 글을 분석하여 다음 정보를 제공해주세요:
item 이라는 이름의 주제와, item content 를 고려하여 답을 찾아 적어주세요.

제약사항:
1. 글 속에 정확하게 포착된 정보만 사용하십시오. 분석 보고서에는 사실 확인이 필요합니다. 처음에는 검증 보고서를 작성한 다음 대화 내용과 비교하여 최종 보고서를 작성하세요.
2. 정확한 답변이 없는 경우 추론하는 대신 'none'으로 답변하십시오.
3. 최대한 찾아서 답변해주세요

질문 내용:
1. 항목: 개인 정보
   항목 내용: 성별, 나이, 교육 (현재 상황, 과거 소속, 학업 고충, 학교 종류, 재입학 이력 등), 거주 상황 및 종교를 포함하세요. 특정하게 식별 가능한 정보를 포함하지 않도록 주의하세요.

2. 항목: 상담 요청 맥락
   항목 내용: 상담 요청으로 이어진 상황을 설명하세요, 특히 자발적으로 상담을 받지 않은 글쓴이의 경우, 상담이 필요한 이유, 받은 지도, 상담의 필요성에 대한 동의 범위 및 외부 압력을 언급하세요.

3. 항목: 주요 걱정 사항
   항목 내용: 글쓴이의 주요 걱정 사항을 그대로 제시하세요. 예를 들어, "정말 압도적이고 더는 어떻게 처리해야 할지 모르겠어요"라고 표현했다면 이를 원본 그대로 포함하세요.



위의 답변들을 동일한 구조를 유지한 채로 아래와 같이 변경해서 다시 한 번 변형해서 저장해주세요:

Response Format: 
{{
    "result": {{
        "q1" : {{"item" : "item", 'answer': "user answer summary"}},
		"q2" : {{"item" : "item", 'answer': "user answer summary"}},
		"q3" : {{"item" : "item", 'answer': "user answer summary"}},
        
    }}
}}

이전 대화를 확인하여 답변을 문맥에 맞게 작성해주세요.
"""

In [54]:
context = [ {'role':'system', 'content':text_analysis_prompt4},
           {'role': 'user', 'content': f"아래의 text를 프롬프트에 기반하여 분석해주세요. {text}"}
          
          ] 
response_raw4, response = get_completion_from_messages(context) 

In [55]:
print(response_raw4)

{"result": {
    "q1": {"item": "개인 정보", "answer": "성별: 불명, 나이: 20대, 교육: 불명, 거주 상황: 불명, 종교: 불명"},
    "q2": {"item": "상담 요청 맥락", "answer": "상담 요청 맥락은 제공되지 않았습니다."},
    "q3": {"item": "주요 걱정 사항", "answer": "글쓴이의 주요 걱정 사항은 제공되지 않았습니다."}
}}


In [56]:
# 심리 검사 분석 프롬프트
text_analysis_prompt5 = """
당신은 심리 분석-GPT입니다. 유명한 심리학자들을 분석하는 데 도움을 주기 위해 설계된 AI입니다.
자신의 강점을 활용하고 법적 문제를 피하기 위해 상대방에게 질문을 하고 그들이 말한 내용을 요약하는 간단한 전략을 가지고 있습니다.


주어진 사용자의 글을 분석하여 다음 정보를 제공해주세요:
item 이라는 이름의 주제와, item content 를 고려하여 답을 찾아 적어주세요.

제약사항:
1. 글 속에 정확하게 포착된 정보만 사용하십시오. 분석 보고서에는 사실 확인이 필요합니다. 처음에는 검증 보고서를 작성한 다음 대화 내용과 비교하여 최종 보고서를 작성하세요.
2. 정확한 답변이 없는 경우 추론하는 대신 'none'으로 답변하십시오.


질문 내용:
4. 항목: 이전 상담 경험
   항목 내용: 글쓴이의 이전 상담 경험을 요약하세요. 상담을 찾은 이유, 그 세션에서 논의한 내용 및 결과에 대한 인식을 포함하세요. 예를 들어, "클라이언트는 학업 스트레스와 가족 갈등으로 인해 이전에 상담을 받았으나, 큰 효과를 보지 못했습니다." 라고 한다면 그렇게 적어주세요.

5. 항목: 글쓴이의 가족 관계
   항목 내용: 가족 구성원과 관계를 화살표로 나타내는 가계도를 만들어보세요. 가족 구성원 (나이, 교육, 직업, 주요 인격 특징 등)에 대한 세부 정보를 제공하세요. 핵심 인물에 대한 글쓴이의 인식, 관계를 적어주세요

6. 항목: 글쓴이의 강점과 자원
   항목 내용: 개인 및 환경 측면에서 보호 요인을 확인하세요. 이러한 요인이 주요 걱정에 긍정적인 영향을 미칠 수 있습니다. 관찰, 인터뷰 또는 심리 검사 결과를 활용하여 이 정보를 수집하세요. 
       예를 들어, "글쓴이의 문제 해결 능력이 강하며 친구들로부터 사회적 지원을 받고 있습니다." 라고 적어주세요.

7. 항목: 글쓴이의 걱정에 대한 대처
   항목 내용: 글쓴이 현재 걱정 사항의 기원과 지속에 대해 설명하세요. 이 걱정 사항을 어떻게 대처했는지, 대처 전략의 효능을 포함하여 이전에 어떻게 대처했는지 상세하게 기술하세요. 종합적인 이해를 위해 가능한 모든 정보를 활용하세요.



위의 답변들을 동일한 구조를 유지한 채로 아래와 같이 변경해서 다시 한 번 변형해서 저장해주세요:

Response Format: 
{{
    "result": {{
        "q4" : {{"item" : "item", 'answer': "user answer summary"}},
		"q5" : {{"item" : "item", 'answer': "user answer summary"}},
		"q6" : {{"item" : "item", 'answer': "user answer summary"}},
        "q7" : {{"item" : "item", 'answer': "user answer summary"}},
        
    }}
}}

이전 대화를 확인하여 답변을 문맥에 맞게 작성해주세요.
"""

In [57]:
context = [ {'role':'system', 'content':text_analysis_prompt5},
           {'role': 'user', 'content': f"아래의 text를 프롬프트에 기반하여 분석해주세요. {text}"}
          
          ] 
response_raw5, response = get_completion_from_messages(context) 

In [58]:
print(response_raw5)

{"result": {
    "q4": {"item": "이전 상담 경험", "answer": "글쓴이는 이전에 상담을 받은 경험이 없습니다."},
    "q5": {"item": "글쓴이의 가족 관계", "answer": "글쓴이는 가족 구성원에 대한 정보를 제공하지 않았습니다."},
    "q6": {"item": "글쓴이의 강점과 자원", "answer": "글쓴이의 강점과 자원에 대한 정보를 제공하지 않았습니다."},
    "q7": {"item": "글쓴이의 걱정에 대한 대처", "answer": "글쓴이의 현재 걱정 사항의 기원과 대처 방법에 대한 정보를 제공하지 않았습니다."}
}}


In [46]:
pd.DataFrame(ast.literal_eval(response_raw5)).T

,q4,q5,q6,q7
result,"{'item': '이전 상담 경험', 'answer': '글쓴이는 이전에 상담을 받...","{'item': '글쓴이의 가족 관계', 'answer': '글쓴이는 가족 구성원에...","{'item': '글쓴이의 강점과 자원', 'answer': '글쓴이의 강점과 자원...","{'item': '글쓴이의 걱정에 대한 대처', 'answer': '글쓴이의 현재 ..."
